# Bert Pruning

[Used the pretained BERT](https://github.com/huggingface/transformers#Quick-tour-TF-20-training-and-PyTorch-interoperability)  
[Reference Post](https://huffon.github.io/2020/03/15/torch-pruning/)

In [2]:
!pip install transformers

     |████████████████████████████████| 573kB 3.4MB/s 
     |████████████████████████████████| 3.7MB 49.4MB/s 
     |████████████████████████████████| 890kB 19.8MB/s 
     |████████████████████████████████| 1.0MB 41.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=d5f0cff7244c06785c7dc132c51bd4a073bc903970b0476ba84200034e056f1e
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


# Local Pruning
> 모델 내 존재하는 텐서들을 개별적으로 가지치기

In [3]:
import torch
import torch.nn.utils.prune as prune
from transformers import BertModel

model = BertModel.from_pretrained('bert-base-cased')

In [15]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [4]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

108310272

In [5]:
module = model.encoder.layer[0].attention.self
print(module)

BertSelfAttention(
  (query): Linear(in_features=768, out_features=768, bias=True)
  (key): Linear(in_features=768, out_features=768, bias=True)
  (value): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
)


In [6]:
module = model.encoder.layer[0].attention.self.key
prune.random_unstructured(module, name="weight", amount=0.3)

print(list(module.named_parameters()))

[('bias', Parameter containing:
tensor([-4.5075e-04,  6.3931e-04, -1.3673e-04, -4.4833e-04, -4.8297e-04,
         8.5335e-04, -6.2471e-04,  7.7209e-04, -4.6402e-04,  4.8441e-04,
         9.7060e-05, -7.4744e-04, -1.4687e-03, -3.6819e-04, -4.6814e-04,
         9.9541e-04, -3.0008e-03, -1.0790e-03, -1.6073e-04, -4.6535e-04,
         5.6987e-04, -1.8128e-03,  2.0990e-03, -3.5929e-05,  2.1796e-04,
         2.0506e-03, -8.8005e-04,  4.4442e-04,  6.4584e-04, -1.9093e-03,
         4.4126e-04,  2.0759e-04, -1.5373e-04, -1.5536e-04,  3.5340e-05,
        -4.4808e-04,  1.3942e-03,  5.9183e-04, -1.4158e-05,  8.4992e-04,
        -7.8199e-04,  2.6565e-04, -1.9708e-03, -8.8281e-04, -9.4142e-04,
        -1.0624e-04, -2.1437e-03,  1.9360e-04, -1.1071e-03, -6.4646e-04,
         5.0200e-04, -7.7265e-04, -3.2641e-04,  3.1411e-05, -1.2639e-03,
         4.9262e-04,  3.7519e-04, -5.6525e-05, -6.5781e-04,  4.6099e-04,
        -2.4043e-03,  7.3155e-04, -1.6624e-03,  4.1991e-04,  5.2776e-06,
        -1.9378e-03

In [7]:
print(list(module.named_buffers()))

[('weight_mask', tensor([[1., 1., 1.,  ..., 0., 1., 1.],
        [0., 1., 0.,  ..., 1., 1., 1.],
        [1., 1., 0.,  ..., 1., 1., 1.],
        ...,
        [0., 1., 1.,  ..., 0., 1., 1.],
        [1., 1., 0.,  ..., 1., 1., 1.],
        [1., 0., 1.,  ..., 1., 1., 1.]]))]


In [8]:
prune.ln_structured(module, name='weight', amount=0.3, n=2, dim=1)
print(list(module.named_buffers()))

[('weight_mask', tensor([[0., 0., 1.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 1., 0., 1.],
        [0., 0., 0.,  ..., 1., 0., 1.],
        ...,
        [0., 0., 1.,  ..., 0., 0., 1.],
        [0., 0., 0.,  ..., 1., 0., 1.],
        [0., 0., 1.,  ..., 1., 0., 1.]]))]


In [9]:
prune.remove(module, 'weight')
print(list(module.named_parameters()))

[('bias', Parameter containing:
tensor([-4.5075e-04,  6.3931e-04, -1.3673e-04, -4.4833e-04, -4.8297e-04,
         8.5335e-04, -6.2471e-04,  7.7209e-04, -4.6402e-04,  4.8441e-04,
         9.7060e-05, -7.4744e-04, -1.4687e-03, -3.6819e-04, -4.6814e-04,
         9.9541e-04, -3.0008e-03, -1.0790e-03, -1.6073e-04, -4.6535e-04,
         5.6987e-04, -1.8128e-03,  2.0990e-03, -3.5929e-05,  2.1796e-04,
         2.0506e-03, -8.8005e-04,  4.4442e-04,  6.4584e-04, -1.9093e-03,
         4.4126e-04,  2.0759e-04, -1.5373e-04, -1.5536e-04,  3.5340e-05,
        -4.4808e-04,  1.3942e-03,  5.9183e-04, -1.4158e-05,  8.4992e-04,
        -7.8199e-04,  2.6565e-04, -1.9708e-03, -8.8281e-04, -9.4142e-04,
        -1.0624e-04, -2.1437e-03,  1.9360e-04, -1.1071e-03, -6.4646e-04,
         5.0200e-04, -7.7265e-04, -3.2641e-04,  3.1411e-05, -1.2639e-03,
         4.9262e-04,  3.7519e-04, -5.6525e-05, -6.5781e-04,  4.6099e-04,
        -2.4043e-03,  7.3155e-04, -1.6624e-03,  4.1991e-04,  5.2776e-06,
        -1.9378e-03

# Global Pruning
> 모델 전체에 Pruning을 한 번에 적용하는 글로벌 Pruning

모델 전체에 글로벌 Pruning을 적용하게 되면 앞선 예제들에서처럼 개별 텐서에서 영향력이 작은 파라미터를 가지치기 하는 것이 아닌, 모듈 간 연결에 있어 영향력이 작은 파라미터들이 가지치기 된다.

In [0]:
final_model = BertModel.from_pretrained('bert-base-cased')

parameters_to_prune = ()
for i in range(12):
    parameters_to_prune += (
        (final_model.encoder.layer[i].attention.self.key, 'weight'),
        (final_model.encoder.layer[i].attention.self.query, 'weight'),
        (final_model.encoder.layer[i].attention.self.value, 'weight'),
    )

prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,
    amount=0.3,
)

In [14]:
for i in range(12):
    print(
        "Sparsity in Layer {}-th key weight: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(final_model.encoder.layer[i].attention.self.key.weight == 0))
            / float(final_model.encoder.layer[i].attention.self.key.weight.nelement())
        )
    )
    print(
        "Sparsity in Layer {}-th query weightt: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(final_model.encoder.layer[i].attention.self.query.weight == 0))
            / float(final_model.encoder.layer[i].attention.self.query.weight.nelement())
        )
    )
    print(
        "Sparsity in Layer {}-th value weight: {:.2f}%".format(
            i+1,
            100. * float(torch.sum(final_model.encoder.layer[i].attention.self.value.weight == 0))
            / float(final_model.encoder.layer[i].attention.self.value.weight.nelement())
        )
    )
    print()

    
numerator, denominator = 0, 0
for i in range(12):
    numerator += torch.sum(final_model.encoder.layer[i].attention.self.key.weight == 0)
    numerator += torch.sum(final_model.encoder.layer[i].attention.self.query.weight == 0)
    numerator += torch.sum(final_model.encoder.layer[i].attention.self.value.weight == 0)

    denominator += final_model.encoder.layer[i].attention.self.key.weight.nelement()
    denominator += final_model.encoder.layer[i].attention.self.query.weight.nelement()
    denominator += final_model.encoder.layer[i].attention.self.value.weight.nelement()
    
print("Global sparsity: {:.2f}%".format(100. * float(numerator) / float(denominator)))

Sparsity in Layer 1-th key weight: 32.36%
Sparsity in Layer 1-th query weightt: 31.65%
Sparsity in Layer 1-th value weight: 42.49%

Sparsity in Layer 2-th key weight: 29.24%
Sparsity in Layer 2-th query weightt: 28.94%
Sparsity in Layer 2-th value weight: 40.62%

Sparsity in Layer 3-th key weight: 28.07%
Sparsity in Layer 3-th query weightt: 27.63%
Sparsity in Layer 3-th value weight: 39.12%

Sparsity in Layer 4-th key weight: 27.86%
Sparsity in Layer 4-th query weightt: 27.64%
Sparsity in Layer 4-th value weight: 37.69%

Sparsity in Layer 5-th key weight: 27.51%
Sparsity in Layer 5-th query weightt: 27.32%
Sparsity in Layer 5-th value weight: 33.67%

Sparsity in Layer 6-th key weight: 25.86%
Sparsity in Layer 6-th query weightt: 25.58%
Sparsity in Layer 6-th value weight: 35.85%

Sparsity in Layer 7-th key weight: 25.83%
Sparsity in Layer 7-th query weightt: 25.23%
Sparsity in Layer 7-th value weight: 36.38%

Sparsity in Layer 8-th key weight: 26.26%
Sparsity in Layer 8-th query weigh